# Candidate Inference Engine

## Detailed Analysis

This script applies the trained machine learning models to real astrophysical data. It is used to infer the nature (Hadronic vs. Quark) of specific neutron star candidates, such as the gravitational wave event GW170817 and massive pulsars like PSR J0740+66.

The script addresses the challenge of observational uncertainty. Since real measurements of Mass, Radius, and Tidal Deformability come with error bars, a single point prediction is insufficient. Instead, a Monte Carlo sampling approach is used to propagate these uncertainties through the classifiers, resulting in a robust, marginalized probability for each candidate.

## Physics and Math

### Monte Carlo Error Propagation
Observations of astrophysical objects provide a central value $\mu$ and an uncertainty $\sigma$ for physical parameters. The script treats these as Gaussian distributions:

$$
x \sim \mathcal{N}(\mu, \sigma^2)
$$

where $x$ represents Mass, Radius, or Tidal Deformability.

### Marginalized Probability
To determine the classification verdict, the probability $P(\text{Quark})$ is integrated over the parameter space defined by the observational errors. This is approximated numerically by averaging the model predictions over $N$ Monte Carlo samples:

$$
P(\text{Quark} | \text{Obs}) \approx \frac{1}{N} \sum_{i=1}^{N} P_{\text{model}}(\text{Quark} | M_i, R_i, \Lambda_i)
$$

This "Soft Voting" mechanism ensures that the final verdict reflects the confidence of the measurement; if the error contours span both Hadronic and Quark regions in the phase space, the resulting probability will reflect that ambiguity.

## Code Walkthrough

### 1. Catalog Definition
A list of candidates is defined, containing the mean values and $1\sigma$ uncertainties for Mass ($M$), Radius ($R$), and Tidal Deformability ($L$).
*   **GW170817**: Includes tidal data ($\Lambda \approx 190$).
*   **Pulsars (e.g., J0740+66)**: Lack tidal measurements ($L=0$), so they rely on Mass and Radius only.

```python
candidates = [
    {"Name": "GW170817", "M": 1.40, "sM": 0.10, "R": 11.90, "sR": 1.40, ...},
    # ...
]
```

### 2. Monte Carlo Sampling
For each candidate, 5000 synthetic realizations are generated using `np.random.normal`.

```python
raw_m = np.random.normal(star['M'], star['sM'], n_mc)
raw_r = np.random.normal(star['R'], star['sR'], n_mc)
raw_l = np.random.normal(star['L'], star['sL'], n_mc)
```

### 3. Physicality Filtering
The Gaussian sampling is truncated to ensure physical realism. Samples with $R < 8$ km or $M < 0.1 M_{\odot}$ are discarded, as these lie outside the domain on which the models were trained (and potentially violate causality or black hole limits).

### 4. Model Selection and Prediction
The script dynamically selects the appropriate classifier based on the available data:
*   **Model A (Full)**: Used if the candidate has tidal data (e.g., GW170817). This model expects inputs `[Mass, Radius, LogLambda]`.
*   **Model Geo**: Used if tidal data is unavailable or zeroed out. This model expects inputs `[Mass, Radius]`.

Probabilities are predicted for all valid samples.

### 5. Verdict Aggregation
The mean probability across all Monte Carlo samples is computed.
*   If $P_{mean} > 0.5$, the object is classified as **QUARK**.
*   Otherwise, it is classified as **HADRONIC**.

The results, including the specific model used and the confidence level, are printed to the console.